In [11]:
import warnings
warnings.filterwarnings(action='ignore')
%config Computer.user_jedi = False

import requests
from bs4 import BeautifulSoup

import time # sleep()

In [12]:
# 1건의 대화 내용을 저장 => 질문과 답변이 한 쌍으로 저장되는 클래스
class Conversation:
    def __init__(self, question, answer): # 생성자 함수. 첫 인수는 self
        self.question = question # 
        self.answer = answer
        
    def __str__(self):
        return f'질문: {self.question}\n답변: {self.answer}'

In [13]:
c =Conversation('누구냐', '내다')
print(c)

질문: 누구냐
답변: 내다


In [14]:
# (대화 주제 + 세부 대화 내용의 url)을 크롤링해 리턴하는 함수
def getSubjects():
    targetSite = 'https://basicenglishspeaking.com/daily-english-conversation-topics/'
    request = requests.get(targetSite)
    html = request.text
    soup=BeautifulSoup(html, 'html.parser')
    
    subjects=[] # 대화 주제
    contentLinks = [] # 세부 대화 내용의 url
    
    divs = soup.findAll('div', {'class':"tcb-flex-col"})
    for div in divs:
        chapter = div.findAll('a') # KEY
        for a in chapter:
            subjects.append(a.text) 
            # contentLinks에 세부 대화 내용의 url을 추가한다
            contentLinks.append(a.get('href'))  # KEY! href속성을 얻어온다.
            
    return subjects, contentLinks # 튜플

In [34]:
subjects, contentLinks = getSubjects()
# for i in range(len(subjects)):
#     print('{:2d}. {}: {}'.format(i + 1, subjects[i], contentLinks[i]))

# 대화 주제에 따른 전체 대화 내용.
conversation = [] # Conversation 클래스 객체를 저장한다.

i = 0 # 대화 주제의 개수를 세는 변수
# 반복하며 대화 주제에 따른 대화 내용을 읽고 저장한다.
for subject in subjects[:1]:
    conversation.append('{:2d}. {}: {}'.format(i + 1, subjects[i], contentLinks[i]))
    
    # 대화 주제별 대화 내용을 크롤링할 페이지를 요청한다.
    request = requests.get(contentLinks[i])
    print(contentLinks[i] + ' 요청 완료') # 로딩중..
    
    # sleep() 메소드는 인수로 지정한 시간만큼 프로그램을 멈춘다. =>초 단위
    # 일정한 간격을 두고 정보를 요청한다.
    time.sleep(1)
    
    html = request.text
    soup = BeautifulSoup(html, 'html.parser')
    
    # 대화 주제 속 대화 내용 : class : sc_player_container1 인 div의 !다음 형제!
    buttons = soup.findAll('div', {'class' : "sc_player_container1" }) # play 버튼 
    print(len(buttons)) # 버튼 개수 : 크롤링할 문장 수
    
    # 각각의 대화 주제에 따른 문장수만큼 반복.
    for button in buttons :
#         print(buttons.index(button))
        # index()가 짝수면 질문, 홀수면 답변
    
        # 크롤링할 문장은 플레이 버튼 내부에 작성된 것이 아니고 다음 형제로 작성되어 있다.
        # next_sibling: 다음 형제, previous_sibling: 이전 형제
        if buttons.index(button) % 2 == 0: # 질문인가?
            print('질문: {}'.format(button.next_sibling))
            question = button.next_sibling
        else:
            print('답변: {}'.format(button.next_sibling))
            answer = button.next_sibling
            # 대화 1건이 완성된다.
        
            # Conversation 클래스 객체를 만드는 작업은 질문과 답변이 한 쌍이 되는 순간 실행한다.
            # 질문과 답변이 한 쌍이 되었으므로 Conversation 클래스 객체를 생성하고 질문과 답변을 저장한다.
            c = Conversation(question, answer)
            # 질문과 답변이 한 쌍으로 저장된 Conversation 클래스 객체를 대화 내용을 기억할 conversation
            # 리스트에 추가한다.
            conversation.append(c)
        # ===== if    
            
    
    # 대화 주제 1건 처리 완료되면 i를 1증가시킨다.
    i+=1

https://basicenglishspeaking.com/family/ 요청 완료
20
질문: How many people are there in your family?
답변: There are 5 people in my family: my father, mother, brother, sister, and me.
질문: Does your family live in a house or an apartment?
답변: We live in a house in the countryside.
질문: What does your father do?
답변: My father is a doctor. He works at the local hospital.
질문: How old is your mother?
답변: She is 40 years old, 1 year younger than my father.
질문: Do you have any siblings? What’s his/her name?
답변: Yes, I do. I have 1 elder brother, David, and 1 younger sister, Mary.
질문: Are you the oldest amongst your brothers and sisters?
답변: No, I’m not. I’m the second child in my family.
질문: What does your mother/father like?
답변: My father likes playing football and my mother likes cooking.
질문: Do your parents let you stay out late?
답변: Of course not. They always ask me to get home before 10 pm each night.
질문: Do you stay with your parents?
답변: Right now, no, but I used to.
질문: Does your family usual

In [33]:
for c in conversation:
    print(c)

 1. Family: https://basicenglishspeaking.com/family/
질문: How many people are there in your family?
답변: There are 5 people in my family: my father, mother, brother, sister, and me.
질문: Does your family live in a house or an apartment?
답변: We live in a house in the countryside.
질문: What does your father do?
답변: My father is a doctor. He works at the local hospital.
질문: How old is your mother?
답변: She is 40 years old, 1 year younger than my father.
질문: Do you have any siblings? What’s his/her name?
답변: Yes, I do. I have 1 elder brother, David, and 1 younger sister, Mary.
질문: Are you the oldest amongst your brothers and sisters?
답변: No, I’m not. I’m the second child in my family.
질문: What does your mother/father like?
답변: My father likes playing football and my mother likes cooking.
질문: Do your parents let you stay out late?
답변: Of course not. They always ask me to get home before 10 pm each night.
질문: Do you stay with your parents?
답변: Right now, no, but I used to.
질문: Does your family us